# 피노봇 메인코드 

피노봇을 실행하기위한 메인 주피터 노트북 코드입니다. 

## 1. 피노봇 부팅 

피노봇을 부팅하고 다이얼로그플로우 서버에 연결하는 코드입니다. 

부팅이 완료되면 OLED 에 "대기중" 메세지가 표시됩니다. 

In [1]:
from modules import pinobot
import time

if __name__ == "__main__":
    bot  = pinobot.PinoBot()
    bot.hardware.write(text="대기중..")

[INFO] (2021-03-24 12:26:55,637 : pinobot.py:106) > [PinoBot] Start Boot
[INFO] (2021-03-24 12:26:55,639 : pino_boot_loader.py:53) > pino_boot_loader.py: start boot
[INFO] (2021-03-24 12:26:55,647 : pino_boot_loader.py:143) > pino_boot_loader.py: config.. Complete


     @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@--@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@@@ ---:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@@----~::@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@@,--,,-~:@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@--,,,.,~:@@@@@::---@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@@--,...,~:@@@@:~~---@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@@----,,,,~:@@@:~-----@@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@------,,,~:@@@:~,,,---@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@@-----,---~~@@:~-.,,,--@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@@@----,...,,,,..~-~-,,,--@@@@@@@@@@@@@@@@@@      
     @@@@@@@@@@@@@@@@@@@@

[INFO] (2021-03-24 12:26:55,969 : v1.py:147) > hardware state : {'I2C': True, 'OLED': True, 'SERVO': True, 'RGB_LED': True, 'SENSOR': True, 'UART': True}
[INFO] (2021-03-24 12:27:01,273 : pino_boot_loader.py:148) > pino_boot_loader.py: hardware.. Complete
[INFO] (2021-03-24 12:27:01,942 : pino_boot_loader.py:354) > pino_boot_loader.py: internet not connected!
[INFO] (2021-03-24 12:27:01,953 : pino_boot_loader.py:159) > pino_boot_loader.py: internet.. Complete
[INFO] (2021-03-24 12:27:05,408 : pino_dialogflow.py:167) > pino_dialogflow.py: sound card index is : 2
[INFO] (2021-03-24 12:27:05,552 : pino_dialogflow.py:359) > pino_dialogflow.py: Start Open Session
[INFO] (2021-03-24 12:27:05,563 : pino_dialogflow.py:374) > pino_dialogflow.py: Open New Session, SID [squarebot01-yauqxoWed Mar 24 12:27:05 2021]
[INFO] (2021-03-24 12:27:06,780 : pinobot.py:113) > [PinoBot] Boot Done..


⚠️ 에러가 발생할 경우, settings/pino_config.ini 의 설정에서 구글 클라우드가 잘 설정됬는지 확인 부탁드립니다. 

## 2. 피노봇 메인루프 


피노봇의 메인 루프문입니다. 

while 문을 반복하면서 

1. 피노봇 상태 업데이트 ( 센서 및 시리얼통신 ) 


2. 초음파 센서가 물체를 감지한경우 

    2.1 음성 스트리밍을 시작합니다 ( 피노봇이 유저의 목소리를 듣기 시작합니다. ) 
    
    2.2 다이얼로그 플로우로부터 응답을 받아서 재생합니다. 
    

3. 시리얼 통신으로 메세지를 수신한경우 
    
    3.1 수신한 메세지를 다이얼로그 플로우로 송신합니다. 
    
    3.2 다이얼로그 플로우로부터 응답을 받아서 재생합니다. 


In [ ]:
    while True:
        
        # 1. 피노봇 센서 및 통신 상태 업데이트
        bot.update()

        # 2.1 초음파 센서가 물체를 감지한경우 
        if bot.state == pinobot.PinoState.SENSOR_ON:

            # 2.2 음성 스트리밍을 시작합니다 ( 피노봇이 유저의 목소리를 듣기 시작합니다. )
            response = bot.listen()

            # 음성 인식에 성공한경우 
            if response is not None:
                bot.hardware.write(text="음성인식 완료!")
                bot.start_say(response)
                bot.start_act(response)
                bot.wait_say_and_act()      # wait until say and act
                print(response.stt_result,"  |  ", response.intent_name,response.intent_response)
                bot.return_idle()           # return to idle state

            # 음성 인식에 실패한경우 
            else :
                bot.hardware.write(text="음성인식 실패 ㅠㅠ")
                time.sleep(3)
                bot.return_idle()

        # 3. 시리얼 통신으로 메세지를 수신한경우
        elif bot.state == pinobot.PinoState.UART_ON:

            # 3.1 수신한 메세지를 다이얼로그 플로우로 송신합니다.
            response = bot.call_uart_event()

            # 응답 수신 성공 
            if response is not None:
                bot.hardware.write(text = "메세지 확인!")
                bot.start_act(response)
                bot.start_say(response)
                bot.wait_say_and_act()      # wait until say and act
                print(response.stt_result, "  |  ", response.intent_name, response.intent_response)
                bot.return_idle()           # return to idle state

            # 응답 수신 실패 
            else:
                bot.hardware.write(text="메세지 잘못보냄 ㅠㅠ")
                time.sleep(0.5)

        time.sleep(0.05) # sleep for 50ms to reduce cpu use

[INFO] (2021-03-24 12:27:11,853 : pinobot.py:175) > listen


get Response....done! 
안뇽   |   Default Fallback Intent 뭐래


KeyboardInterrupt: 